In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')

In [9]:
df ['days_employed'] = df ['days_employed'].abs()
median_days = df['days_employed'].median()
print (median_days)

2194.220566878695


In [11]:
df['days_employed'] = df['days_employed'].fillna(median_days)

In [18]:
median_income = df['total_income'].median ()
print (median_income)

145017.93753253992


In [19]:
df['total_income'] = df['total_income'].fillna(median_income)

In [22]:
df['total_income'] = df ['total_income'].astype(int)

In [33]:
print (df.duplicated().sum())

0


In [32]:
df = df.drop_duplicates().reset_index (drop = True)

In [29]:
df ['education'] = df ['education'].str.lower()

In [35]:
print (df.head(10))

   children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   среднее             1   
3         3    4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   среднее             1   
8         2    6929.865299         35    высшее             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

In [36]:
education_dict = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

In [38]:
family_status_dict = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

In [39]:
df = df.drop(columns=['education', 'family_status'])

In [40]:
def categorize_income(income):
    if 0 <= income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif income >= 1000001:
        return 'A'

In [41]:
df['total_income_category'] = df['total_income'].apply(categorize_income)

In [42]:
print(df['total_income_category'].value_counts())

total_income_category
C    16016
B     5041
D      350
A       25
E       22
Name: count, dtype: int64


In [44]:
def categorize_purpose(purpose):
    # Важно: ищем подстроки в нижнем регистре
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жил' in purpose:
        return 'операции с недвижимостью'
    elif 'свад' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'
    else:
        return 'другое' 

df['purpose_category'] = df['purpose'].apply(categorize_purpose)


print(df['purpose_category'].value_counts())

purpose_category
операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: count, dtype: int64


In [45]:
# Создаем функцию для автоматизации ответов, чтобы не писать одно и то же
def make_pivot(df, index_column):
    pivot = df.pivot_table(index=index_column, values='debt', aggfunc=['count', 'sum', 'mean'])
    pivot.columns = ['Всего клиентов', 'Кол-во должников', 'Доля просрочки']
    # Сортируем по доле просрочки для наглядности
    return pivot.sort_values(by='Доля просрочки', ascending=False)

# 1. Зависимость от количества детей
print(make_pivot(df, 'children'))

# 2. Зависимость от семейного положения (используем созданный словарь или ID)
# Если ты уже удалил текст, используй family_status_id и приклей словарь обратно, 
# либо используй данные до удаления текста.
print(make_pivot(df, 'family_status_id')) 

# 3. Зависимость от уровня дохода
print(make_pivot(df, 'total_income_category'))

# 4. Зависимость от целей кредита
print(make_pivot(df, 'purpose_category'))

          Всего клиентов  Кол-во должников  Доля просрочки
children                                                  
 20                   76                 8        0.105263
 4                    41                 4        0.097561
 2                  2052               194        0.094542
 1                  4808               444        0.092346
 3                   330                27        0.081818
 0                 14091              1063        0.075438
-1                    47                 1        0.021277
 5                     9                 0        0.000000
                  Всего клиентов  Кол-во должников  Доля просрочки
family_status_id                                                  
4                           2810               274        0.097509
1                           4151               388        0.093471
0                          12339               931        0.075452
3                           1195                85        0.071130
2       

In [46]:
print(df.head(10))

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1    8437.673028         42             0                 0      F   
1         1    4024.803754         36             1                 0      F   
2         0    5623.422610         33             1                 0      M   
3         3    4124.747207         32             1                 0      M   
4         0  340266.072047         53             1                 1      F   
5         0     926.185831         27             0                 1      M   
6         0    2879.202052         43             0                 0      F   
7         0     152.779569         50             1                 0      M   
8         2    6929.865299         35             0                 1      F   
9         0    2188.756445         41             1                 0      M   

  income_type  debt  total_income                     purpose  \
0   сотрудник     0        253875               покупк